In [23]:
import numpy as np
from tensorflow.keras.datasets import mnist


# Linear Layer Class
class Linear:
    def __init__(self, input_dim, output_dim):
        """
        Initializes a fully connected layer with weights and biases.
        """
        self.weights = np.random.randn(input_dim, output_dim) * 0.01
        self.biases = np.zeros((1, output_dim))
        self.input = None
        self.output = None

    def forward(self, x):
        """
        Performs the forward pass through the layer.
        """
        self.input = x
        self.output = np.dot(x, self.weights) + self.biases
        return self.output

    def backward(self, d_out):
        """
        Performs the backward pass, updating the weights and biases.
        """
        self.d_weights = np.dot(self.input.T, d_out)
        self.d_biases = np.sum(d_out, axis=0, keepdims=True)
        d_input = np.dot(d_out, self.weights.T)

        return d_input

# ReLU Activation Class
class ReLU:
    def forward(self, x):
        """
        Applies the ReLU activation function.
        """
        self.input = x
        return np.maximum(0, x)

    def backward(self, d_out):
        """
        Computes the gradient of the ReLU function.
        """
        return d_out * (self.input > 0)

# Sigmoid Activation Class
class Sigmoid:
    def forward(self, x):
        """
        Applies the Sigmoid activation function.
        """
        self.input = x
        return 1 / (1 + np.exp(-x))

    def backward(self, d_out):
        """
        Computes the gradient of the Sigmoid function.
        """
        sigmoid = self.forward(self.input)
        return d_out * sigmoid * (1 - sigmoid)

# Tanh Activation Class
class Tanh:
    def forward(self, x):
        """
        Applies the Tanh activation function.
        """
        self.input = x
        return np.tanh(x)

    def backward(self, d_out):
        """
        Computes the gradient of the Tanh function.
        """
        return d_out * (1 - np.tanh(self.input) ** 2)

# Softmax Activation Class
class Softmax:
    def forward(self, x):
        """
        Applies the Softmax activation function.
        """
        exp_values = np.exp(x - np.max(x, axis=1, keepdims=True))
        self.output = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        return self.output

    def backward(self, d_out):
        """
        Placeholder for backward pass, typically used with Cross-Entropy Loss.
        """
        return d_out  # Simplified for use with Cross-Entropy Loss

# Cross-Entropy Loss Class
class CrossEntropyLoss:
    def forward(self, y_pred, y_true):
        """
        Computes the forward pass of the cross-entropy loss.
        """
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        correct_confidences = y_pred_clipped[range(samples), y_true]
        loss = -np.log(correct_confidences)
        return np.mean(loss)

    def backward(self, y_pred, y_true):
        """
        Computes the gradient of the cross-entropy loss.
        """
        samples = len(y_pred)
        y_pred[range(samples), y_true] -= 1
        return y_pred / samples

# Mean Squared Error (MSE) Loss Class
class MSELoss:
    def forward(self, y_pred, y_true):
        """
        Computes the forward pass of the MSE loss.
        """
        return np.mean((y_pred - y_true) ** 2)

    def backward(self, y_pred, y_true):
        """
        Computes the gradient of the MSE loss.
        """
        return 2 * (y_pred - y_true) / y_true.size

# SGD Optimizer Class
class SGD:
    def __init__(self, learning_rate=0.01):
        """
        Initializes the stochastic gradient descent optimizer.
        """
        self.learning_rate = learning_rate

    def step(self, layers):
        """
        Updates the weights and biases of each layer.
        """
        for layer in layers:
            if hasattr(layer, 'weights'):
                layer.weights -= self.learning_rate * layer.d_weights
                layer.biases -= self.learning_rate * layer.d_biases

# Model Class
class Model:
    def __init__(self):
        """
        Initializes the model with a list of layers.
        """
        self.layers = []
        self.loss = None
        self.optimizer = None

    def add_layer(self, layer):
        """
        Adds a layer to the model.
        """
        self.layers.append(layer)

    def compile(self, loss, optimizer):
        """
        Compiles the model with a loss function and optimizer.
        """
        self.loss = loss
        self.optimizer = optimizer

    def forward(self, x):
        """
        Performs the forward pass through all layers.
        """
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, d_out):
        """
        Performs the backward pass through all layers.
        """
        for layer in reversed(self.layers):
            d_out = layer.backward(d_out)

    def train(self, x_train, y_train, epochs=20, batch_size=64):
        """
        Trains the model on the given training data.
        """
        for epoch in range(epochs):
            for i in range(0, len(x_train), batch_size):
                x_batch = x_train[i:i + batch_size]
                y_batch = y_train[i:i + batch_size]

                y_pred = self.forward(x_batch)
                loss = self.loss.forward(y_pred, y_batch)

                d_out = self.loss.backward(y_pred, y_batch)
                self.backward(d_out)
                self.optimizer.step(self.layers)

            print(f'Epoch {epoch + 1}, Loss: {loss}')

    def predict(self, x):
        """
        Predicts output using the trained model.
        """
        return self.forward(x)

    def evaluate(self, x_test, y_test):
        """
        Evaluates the model on the test data.
        """
        y_pred = self.forward(x_test)
        loss = self.loss.forward(y_pred, y_test)
        accuracy = np.mean(np.argmax(y_pred, axis=1) == y_test)
        return loss, accuracy

    def save(self, filepath):
        """
        Saves the model weights to a file.
        """
        model_params = [layer.__dict__ for layer in self.layers]
        np.save(filepath, model_params)

    def load(self, filepath):
        """
        Loads the model weights from a file.
        """
        model_params = np.load(filepath, allow_pickle=True)
        for layer, params in zip(self.layers, model_params):
            layer.__dict__.update(params)


In [24]:
# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], -1) / 255.0
x_test = x_test.reshape(x_test.shape[0], -1) / 255.0

# Define the model
model = Model()
model.add_layer(Linear(784, 128))
model.add_layer(ReLU())
model.add_layer(Linear(128, 10))
model.add_layer(Softmax())

# Compile the model
loss = CrossEntropyLoss()
optimizer = SGD(learning_rate=0.01)
model.compile(loss, optimizer)

# Train the model
model.train(x_train, y_train, epochs=20, batch_size=64)

# Save the model
model.save('mnist_model.npy')



Epoch 1, Loss: 0.9612261202205985
Epoch 2, Loss: 0.38328471116218316
Epoch 3, Loss: 0.25497216401449396
Epoch 4, Loss: 0.20014453996888873
Epoch 5, Loss: 0.16996188909791965
Epoch 6, Loss: 0.15011477161080464
Epoch 7, Loss: 0.13603497095030595
Epoch 8, Loss: 0.1251693485829672
Epoch 9, Loss: 0.1169721432562974
Epoch 10, Loss: 0.10908072997560189
Epoch 11, Loss: 0.10153310872373333
Epoch 12, Loss: 0.09482178891877731
Epoch 13, Loss: 0.08927320163778878
Epoch 14, Loss: 0.08439665179697202
Epoch 15, Loss: 0.08019829885025592
Epoch 16, Loss: 0.07628635914533811
Epoch 17, Loss: 0.07250012880212206
Epoch 18, Loss: 0.06928022849297849
Epoch 19, Loss: 0.06613621374648163
Epoch 20, Loss: 0.06334478948827633


In [25]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

# Load the model (for demonstration purposes)
model.load('mnist_model.npy')

Test Loss: 0.19872000453163513, Test Accuracy: 0.941
